simulate the two player farm game

first, set up big config dictionary that specifies the game to be played

jan 2023


In [10]:
# imports
from random import randint
import numpy as np
import random
import datetime # for limiting calculation to wall clock time
import math
import copy
import matplotlib.pyplot as plt
import csv
import sys
import pandas as pd
import farmgame
from mcts import MCTS
from agents import RandomPolicy


In [6]:
#Set params
layers = ["Items00", "Items01", "Items02", "Items03", "Items04", "Items05", "Items06", "Items07", "Items08", "Items09", "Items10", "Items11"]
resourceConds = ["even", "uneven"]
costCond="low"
visibilityCond="full"
redFirst=True

In [13]:
gameNum = 0
game_df = {"gameNum": [], "objectLayer": [], "resourceCond": [], "costCond": [], "visibilityCond": [], "redFirst": [], "player": [], "action": [], "veg_color": [], "steps": [], "helpful": [], "redRwd": [], "purpRwd": []}

for resourceCond in resourceConds:
    for layer in layers:
        print(f"{resourceCond} game with layer {layer}")
        TheFarm = farmgame.configure_game(layer=layer,resourceCond=resourceCond,costCond=costCond,visibilityCond=visibilityCond,redFirst=redFirst)
        state = TheFarm

        done = False

        # red
        red_agent = MCTS(time=2., C=1, max_moves = 20, color="red")
        # red_agent = RandomPolicy(game)
        
        # purple
        purp_agent = MCTS(time=2., C=1, max_moves = 20, color="purple")
        # purp_agent = RandomPolicy(game)

        action_seq = []
        # print("current farm items: " + str(list(v.name for v in state.items if v.status=="farm")))
        # print("current box: " + str(list(v.name for v in state.farmbox.contents)))
        
        t = 0
        while not done:
            red_agent.update(state) 
            purp_agent.update(state)

            currentplayer = state.players[state.turn]
            
            if currentplayer['name'] == "red":
                action = red_agent.choose_action()
            else:
                action = purp_agent.choose_action()
            
            #find out if action is helpful or not
            transition = farmgame.Transition(state,action)
            helpful = transition.is_helping()
        
            #get steps taken
            steps = state.get_steps(action)
            
            if action is None:
                print("THIS SHOULdn't happen")
                print(currentplayer['name'] + " has no more moves.")
            else:
                print("turn %s complete"%(t))

            state = state.take_action(action,inplace=True) #pick first veg in list
            rwd, done = state.reward(currentplayer['name'])
        
            #append each turn's action data to dictionary and increment turn number
            game_df["gameNum"].append(gameNum)
            game_df["objectLayer"].append(layer)
            game_df["resourceCond"].append(resourceCond)
            game_df["costCond"].append(costCond)
            game_df["visibilityCond"].append(visibilityCond)
            game_df["redFirst"].append(redFirst)
            game_df["player"].append(currentplayer['name'])
            game_df["action"].append(action.name) 
            game_df["veg_color"].append(action.color) 
            game_df["steps"].append(steps)
            game_df["helpful"].append(helpful)
            t += 1
        
        #append the rewards at the end of the game to dictionary and increment game number
        game_df["redRwd"] += ([state.reward('red')[0]] * t)
        game_df["purpRwd"] += ([state.reward('purple')[0]] * t)
        
        print("game %s done"%(gameNum))
        gameNum += 1
        

even game with layer Items00
turn 0 complete
turn 1 complete
turn 2 complete
turn 3 complete
turn 4 complete
turn 5 complete
turn 6 complete
turn 7 complete
turn 8 complete
turn 9 complete
game 0 done
even game with layer Items01
turn 0 complete
turn 1 complete
turn 2 complete
turn 3 complete
turn 4 complete
turn 5 complete
turn 6 complete
turn 7 complete
turn 8 complete
turn 9 complete
game 1 done
even game with layer Items02
turn 0 complete
turn 1 complete
turn 2 complete
turn 3 complete
turn 4 complete
turn 5 complete
turn 6 complete
turn 7 complete
turn 8 complete
turn 9 complete
turn 10 complete
turn 11 complete
game 2 done
even game with layer Items03
turn 0 complete
turn 1 complete
turn 2 complete
turn 3 complete
turn 4 complete
turn 5 complete
turn 6 complete
turn 7 complete
turn 8 complete
turn 9 complete
game 3 done
even game with layer Items04
turn 0 complete
turn 1 complete
turn 2 complete
turn 3 complete
turn 4 complete
turn 5 complete
turn 6 complete
turn 7 complete
turn 

In [14]:
#function to intake the long dataframe with all games and all actions and return each game's relevant output metrics
def aggregate_game_outputs(game_df):
    helping_event = game_df.helpful.sum()
    steps = game_df.steps.sum()
    helpful_steps = game_df.loc[game_df.helpful,"steps"].sum()
    helped = helping_event > 0 
    purpleHelpingEvents = game_df.loc[game_df.player == "purple","helpful"].sum()
    redHelpingEvents = game_df.loc[game_df.player == "red","helpful"].sum()
    purpleHelped = purpleHelpingEvents > 0 
    redHelped = redHelpingEvents > 0 
    purpleHelpfulSteps = game_df.loc[(game_df.player == "purple") & (game_df.helpful),"steps"].sum()
    redHelpfulSteps = game_df.loc[(game_df.player == "red") & (game_df.helpful),"steps"].sum()
    nRedVeg = (game_df.veg_color == "red").sum()
    nPurpVeg = (game_df.veg_color == "purple").sum()
    
    agg_mets = pd.Series({"helping_event":helping_event, "steps": steps, "helpful_steps": helpful_steps, "helped": helped, "purpleHelpingEvents": purpleHelpingEvents,
                          "redHelpingEvents": redHelpingEvents, "purpleHelped": purpleHelped, "redHelped": redHelped, "purpleHelpfulSteps": purpleHelpfulSteps, 
                          "redHelpfulSteps": redHelpfulSteps, "nRedVeg": nRedVeg, "nPurpVeg": nPurpVeg}) 
    return agg_mets

In [16]:
game_df = pd.DataFrame(game_df)
game_df_results = game_df.groupby(["gameNum", "objectLayer", "resourceCond", "costCond", "visibilityCond", "redFirst"]).apply(aggregate_game_outputs)
game_df_results

/var/folders/z0/1hhlpnfn2mz3vk7t57zdcy0w0000gn/T/ipykernel_53729/1662983422.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  game_df_results = game_df.groupby(["gameNum", "objectLayer", "resourceCond", "costCond", "visibilityCond", "redFirst"]).apply(aggregate_game_outputs)


,,,,,,helping_event,steps,helpful_steps,helped,purpleHelpingEvents,redHelpingEvents,purpleHelped,redHelped,purpleHelpfulSteps,redHelpfulSteps,nRedVeg,nPurpVeg
gameNum,objectLayer,resourceCond,costCond,visibilityCond,redFirst,,,,,,,,,,,,
0,Items00,even,low,full,True,6,77,30,True,3,3,True,True,15,15,4,4
1,Items01,even,low,full,True,4,99,50,True,2,2,True,True,24,26,4,4
2,Items02,even,low,full,True,1,80,6,True,1,0,True,False,6,0,6,3
3,Items03,even,low,full,True,5,77,29,True,2,3,True,True,13,16,3,5
4,Items04,even,low,full,True,11,161,72,True,5,6,True,True,31,41,7,9
5,Items05,even,low,full,True,8,171,70,True,4,4,True,True,23,47,7,7
6,Items06,even,low,full,True,8,57,32,True,4,4,True,True,16,16,4,4
7,Items07,even,low,full,True,8,78,49,True,4,4,True,True,24,25,5,4
8,Items08,even,low,full,True,1,69,12,True,0,1,False,True,0,12,3,5
